### import libraries

In [6]:
import pandas as pd
import seaborn as sns
import requests
import requests.auth
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import praw
import pickle
import time
import urllib, json
from urllib3.util import Retry
from datetime import timedelta,datetime
import logging

# logger = logging.getLogger(__name__)

### Define needed functions

In [7]:
def save_pickle(file_name,obj):
    with open(file_name, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return file_name + ' is saved'

def open_pickle(file_name):
    with open(file_name, 'rb') as handle:
        obj = pickle.load(handle)
    return obj

def get_credentials(path):
    
    with open(path, 'rb') as handle:
         cred = pickle.load(handle)
    
    return cred

def get_access_token_headers(credentials):
    client_auth = requests.auth.HTTPBasicAuth(credentials['client_id'], 
                                              credentials['client_secret'])
    post_data = {"grant_type": "password", "username": credentials['username'],
                 "password": credentials['password']}
    headers = {"User-Agent": credentials['user_agent']}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
    token = 'bearer '+response.json()['access_token']
    return {"Authorization": token, "User-Agent": credentials['user_agent']}

    
def retrieve_comment_and_post_count(reddit_data_dict,subreddit):
    lookback_hours = 24

    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
        }
    stats = {'comment':'num_of_comments', 'submission':'num_of_posts'}
    url = "https://api.pushshift.io/reddit/{}/search?&limit=1000&sort=desc&subreddit={}&before="

    startTime = datetime.utcnow()
    endTime = startTime - timedelta(hours=lookback_hours)
    endEpoch = int(endTime.timestamp())

    for stat in stats:
        count = 0
        breakOut = False
        previousEpoch = int(startTime.timestamp())
        while True:
            newUrl = url.format(stat, subreddit)+str(previousEpoch)
            json = requests.get(newUrl, headers=headers)
            objects = json.json()['data']
            for obj in objects:
                previousEpoch = obj['created_utc'] - 1
                if previousEpoch < endEpoch:
                    breakOut = True
                    break
                count += 1

            if breakOut:
                reddit_data_dict[stats[stat]].append(count)
                break


def requests_retry_session(retries=3,
                           backoff_factor=0.3,
                           status_forcelist=(500, 502, 503, 504),
                           session=None,):
    session = session or requests.Session()
    retry = Retry(total=retries,
                  read=retries,
                  connect=retries,
                  backoff_factor=backoff_factor,
                  status_forcelist=status_forcelist,)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


def call_api(url, headers,sub, timeout=10):
#     payload = {}
    try:
        response = requests_retry_session().get(url, headers=headers, timeout=timeout)
        response.encoding = 'utf-8'
        if response.status_code != 401:
            response.raise_for_status()
    except (requests.exceptions.Timeout, requests.exceptions.HTTPError):
#         logger.error("Error calling api with payload %s", payload, exc_info=True)
        raise Exception('API response: {}'.format(response.status_code))
    return response

def add_data_to_dict(reddit_data_dict,time_stamp,sub,file_name,headers):
#     payload = {'subreddit':sub, 'time_stamp':time_stamp}
    #rate limit is one request per second (60 requests per min)
    start_time = time.time()
    
    subreddit = call_api(f"https://oauth.reddit.com/r/{sub}/about", headers, sub, timeout=10)
    reddit_data_dict['date'].append(current_timestamp)
    reddit_data_dict['title'].append(subreddit.json()['data']['title'])
    reddit_data_dict['id'].append(subreddit.json()['data']['id'])
    reddit_data_dict['subscribers'].append(subreddit.json()['data']['subscribers'])
    retrieve_comment_and_post_count(reddit_data_dict,sub)
    
    save_pickle(file_name, reddit_data_dict)
    sleep_time = 5 - (time.time() - start_time)
    if sleep_time < 0:
        sleep_time = 0.5
    time.sleep(sleep_time)

def parse_recent_activity(current_timestamp, credentials, subreddit_names):
    now = datetime.now().strftime("%Y_%m_%d")
    file_name = 'subreddit_scrape_' + now + '_dict.pkl'
    
    reddit_data = {
        'date':[],
        'title':[],
        'id':[],
        'subscribers':[],
        'num_of_comments':[],
        'num_of_posts':[] 
    }
    
    
    auth_header = get_access_token_headers(credentials)
    
    skipped = []
    
    c = 0
    for sub in subreddit_names:
        
        c += 1
        try:
            print(sub)
            add_data_to_dict(reddit_data,current_timestamp,sub,file_name,auth_header)
            
            if c == 1000:
                c = 0
                get_access_token_headers(path)
        except Exception as err:
            print(err)
            skipped.append(sub)
            save_pickle('skipped_'+file_name, skipped)
            pass
        
        
    return reddit_data

In [8]:
now = datetime.now()
current_timestamp = now.strftime("%Y-%m-%d")
credentials = get_credentials('reddit_credentials.pkl')

df = open_pickle('data/game_and_subreddit_pairing_05072020.pkl')

subreddit_names = df['subreddit']

parse_recent_activity(current_timestamp, credentials, subreddit_names.unique())

AnthemTheGame
LowSodiumAnthem
AnthemMemes
ARK
playark
ARK_pc
ARKone
assassinscreed
assassinscreed3
arkhamgames
BatmanArkham
arkhamknight
batmanarkhamorigins
Battleborn
Battlefield1_PS4
battlefield_live
battlefield_comp
battlefield_one
battlefield1
Battlefield
battlefield3
battlefield_4
battlefield4
API response: 403
battlefieldhardline
BF_Hardline
Battlefield_Hardline
BattlefieldV
BioshockInfinite
Bioshock
None
bloodbornethegame
bloodborne
bloodbornesliders
BloodborneLore
Bloodstained
Borderlands
Borderlands2
borderlands
BorderlandsPreSequel
borderlandstps
CodAW
CallOfDuty
blackops2
blackops3
CoDCompetitive
CODGhosts
COD_LFG
Infinitewarfare
MW2
ModernWarfareGame
mw4
modernwarfare
CODModernWarfare
API response: 403
mw3
Cod4Remastered
CallOfDutyWorldWarTwo
WWII
Call_of_Duty_WWII
CitiesSkylinesModding
CitiesSkylines
XboxCitiesSkylines
ConanBaseBuilds
Conan_Exiles
ConanExiles
counterstrike
GlobalOffensiveTrade
GlobalOffensive
csgo
crashbandicoot
crashteamracing
CrusaderKings
crusaderkings2

reddeadredemption
Expecting value: line 1 column 1 (char 0)
reddead
Expecting value: line 1 column 1 (char 0)
reddeadmemes
Expecting value: line 1 column 1 (char 0)
PCRedDead
Expecting value: line 1 column 1 (char 0)
reddeadfashion
Expecting value: line 1 column 1 (char 0)
RedDeadOnline
Expecting value: line 1 column 1 (char 0)
rdr2online
Expecting value: line 1 column 1 (char 0)
reddeadredemption2_pc
Expecting value: line 1 column 1 (char 0)
Red_Dead_Redemption_2
Expecting value: line 1 column 1 (char 0)
reddeadredemption2
Expecting value: line 1 column 1 (char 0)
RDR2
Expecting value: line 1 column 1 (char 0)
RedDeadRedemption_2
Expecting value: line 1 column 1 (char 0)
ResidentEvil2Remake
Expecting value: line 1 column 1 (char 0)
residentevil
Expecting value: line 1 column 1 (char 0)
RESIDENTEVII
Expecting value: line 1 column 1 (char 0)
TombRaider
Expecting value: line 1 column 1 (char 0)
riseofthetombraider
Expecting value: line 1 column 1 (char 0)
RocketLeagueSchool
Expecting val

Totalwarwarhammer
Expecting value: line 1 column 1 (char 0)
totalwarhammer
Expecting value: line 1 column 1 (char 0)
transformers
Expecting value: line 1 column 1 (char 0)
TransformersFOC
Expecting value: line 1 column 1 (char 0)
StreetFighter
Expecting value: line 1 column 1 (char 0)
SF4
Expecting value: line 1 column 1 (char 0)
Uncharted_4
Expecting value: line 1 column 1 (char 0)
unchartedmultiplayer
Expecting value: line 1 column 1 (char 0)
uncharted
Expecting value: line 1 column 1 (char 0)
Uncharted4
Expecting value: line 1 column 1 (char 0)
UC4
Expecting value: line 1 column 1 (char 0)
Vermintide
Expecting value: line 1 column 1 (char 0)
Vermintide2
Expecting value: line 1 column 1 (char 0)
watch_dogs
Expecting value: line 1 column 1 (char 0)
Watch_Dogs_Two
Expecting value: line 1 column 1 (char 0)
WatchDogs2Game
Expecting value: line 1 column 1 (char 0)
Wolfenstein
Expecting value: line 1 column 1 (char 0)
worldwarz
Expecting value: line 1 column 1 (char 0)
worldwarzthegame
Exp

Expecting value: line 1 column 1 (char 0)
DivinityGameMaster
Expecting value: line 1 column 1 (char 0)
DOS2GM
Expecting value: line 1 column 1 (char 0)
DivinityOriginalSin2
Expecting value: line 1 column 1 (char 0)
DivinityOriginalSin
Expecting value: line 1 column 1 (char 0)
hwl
Expecting value: line 1 column 1 (char 0)
wwe2k16
Expecting value: line 1 column 1 (char 0)
WWEGame
Expecting value: line 1 column 1 (char 0)
WWE2K16Mods
Expecting value: line 1 column 1 (char 0)
TeamSonicRacers
Expecting value: line 1 column 1 (char 0)
TeamSonicRacing
Expecting value: line 1 column 1 (char 0)
30YearsOfMadden
Expecting value: line 1 column 1 (char 0)
swagmemes4madden
Expecting value: line 1 column 1 (char 0)
MaddenNFL17CheapCoins
Expecting value: line 1 column 1 (char 0)
GuitarHero
Expecting value: line 1 column 1 (char 0)
guitarherolive
Expecting value: line 1 column 1 (char 0)
FarCryPrimalCrack
Expecting value: line 1 column 1 (char 0)
FarCryPrimal
Expecting value: line 1 column 1 (char 0)
P

Expecting value: line 1 column 1 (char 0)
civcast
Expecting value: line 1 column 1 (char 0)
NHL20_EASHL
Expecting value: line 1 column 1 (char 0)
NHL20
Expecting value: line 1 column 1 (char 0)
NHL20Tournaments
Expecting value: line 1 column 1 (char 0)
EASHL
Expecting value: line 1 column 1 (char 0)
MBAlive
Expecting value: line 1 column 1 (char 0)
ea_ufc
Expecting value: line 1 column 1 (char 0)
easportsufc3
Expecting value: line 1 column 1 (char 0)
UFC3
Expecting value: line 1 column 1 (char 0)
Madden_NGEL
Expecting value: line 1 column 1 (char 0)
Ballcoins
Expecting value: line 1 column 1 (char 0)
CouchFootballLeague
Expecting value: line 1 column 1 (char 0)
Madden18
Expecting value: line 1 column 1 (char 0)
MutSquad
Expecting value: line 1 column 1 (char 0)
NONE
Expecting value: line 1 column 1 (char 0)
kingdomheartsspoilers
Expecting value: line 1 column 1 (char 0)
KH3
Expecting value: line 1 column 1 (char 0)
KingdomHearts3
Expecting value: line 1 column 1 (char 0)
TheKingdomsHea

{'date': ['2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',
  '2020-05-21',


In [21]:
df_2 = open_pickle('subreddit_scrape_2020_05_14_dict.pkl')

In [22]:
df_2

{'date': ['2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
  '2020-05-14',
